In [1]:
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright
import time
import numpy as np
pd.set_option('display.max_colwidth', None)

In [2]:
# Take today's date and format it - must be used in the URLs.

date = date.today()

date = date.strftime("%d.%m.%Y")

print(date)

27.01.2023


In [3]:
# Read in the links and update with today's date
df = pd.read_csv("data/links.csv")

df = df.replace(to_replace=r'01.01.2023', value= date, regex=True)

In [4]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

url = df.iloc[0]['link']
await page.goto(url)

time.sleep(1)
await page.locator("#close-button").click()

In [5]:
# Create an empty dataframe for the scrape.
total_df = pd.DataFrame()

In [6]:
for index, row in df.iterrows():
    url = row['link']

    country = row['country']
    area = row['area']
    
    await page.goto(url)
    time.sleep(5)
    await page.get_by_text("PT60M").click()

    time.sleep(5)

    tables = pd.read_html(await page.content(), flavor = 'html5lib')
    data = tables[0]
    data.insert(0, 'country', country)
    data.insert(1, 'area', area)
    total_df = pd.concat([total_df, data], axis=0)

In [23]:
total_df

country          area            MTU  Day-ahead Price              \
                                            MTU [Currency / MWh] [EUR / MWh]   
0          Albania        BZN AL  00:00 - 01:00              NaN         NaN   
1          Albania        BZN AL  01:00 - 02:00              NaN         NaN   
2          Albania        BZN AL  02:00 - 03:00              NaN         NaN   
3          Albania        BZN AL  03:00 - 04:00              NaN         NaN   
4          Albania        BZN AL  04:00 - 05:00              NaN         NaN   
..             ...           ...            ...              ...         ...   
19  United Kingdom  BZN GB(IFA2)  19:00 - 20:00              n/e         NaN   
20  United Kingdom  BZN GB(IFA2)  20:00 - 21:00              n/e         NaN   
21  United Kingdom  BZN GB(IFA2)  21:00 - 22:00              n/e         NaN   
22  United Kingdom  BZN GB(IFA2)  22:00 - 23:00              n/e         NaN   
23  United Kingdom  BZN GB(IFA2)  23:00 - 00:00              n/e         NaN   

                      date  
   [UAH / MWh]              
0          NaN  01/27/2023  
1          NaN  01/27/2023  
2          NaN  01/27/2023  
3          NaN  01/27/2023  
4          NaN  01/27/2023  
..         ...         ...  
19         NaN  01/27/2023  
20         NaN  01/27/2023  
21         NaN  01/27/2023  
22         NaN  01/27/2023  
23         NaN  01/27/2023  

[1896 rows x 7 columns]

In [22]:
total_df['date'] = pd.to_datetime('today').strftime("%m/%d/%Y")

total_df.columns = total_df['country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)', 'date']

# Standardize the missing values
total_df.replace({'-': np.nan}, inplace=True)
total_df.replace({'n/e': np.nan}, inplace=True)

# And reorder the columns
all_data = total_df[['date', 'country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)']]

KeyError: 'Key length (6) exceeds index depth (2)'

In [ ]:
all_data.to_csv("data/europaenergy_today.csv", index=False)